In [ ]:
paths = {"out": []}
with open("input.txt", "r") as f:
# with open("example_input.txt", "r") as f:
    for line in f:
        devices = line.strip().split(' ')
        paths[devices[0][:-1]] = devices[1:]

In [ ]:
# %%timeit 175 μs ± 7.97 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
# part 1 
def dfs(start, finish, tree, sub_path, allPaths):
    sub_path.append(start)
    if start == finish:
        # print(sub_path)
        allPaths.append(sub_path.copy())
    else:
        for child_node in tree[start]:
            dfs(child_node, finish, tree, sub_path, allPaths)
    
    sub_path.pop()
    return allPaths

print("part 1 : ", len(dfs('you', 'out', paths, [], [])))

In [ ]:
# %%timeit 1.05 ms ± 21.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
# part 1 with polars
import polars as pl

def count_paths(paths_dict: dict, start: str, target: str) -> int:
    # Convert dict → edges DF
    edges = pl.DataFrame({
        "parent": [p for p, childs in paths_dict.items() for _ in childs],
        "child":  [c for p, childs in paths_dict.items() for c in childs],
    })
    level = pl.DataFrame({"last": [start]})
    all_levels = [level]
    max_depth = len(paths_dict)

    for _ in range(max_depth):
        next_level = (
            level.join(edges, left_on="last", right_on="parent")
                 .select(pl.col("child").alias("last"))
        )
        if next_level.is_empty():
            break

        all_levels.append(next_level)
        level = next_level

    reachable = pl.concat(all_levels)
    return reachable.filter(pl.col("last") == target).shape[0]

# part 1
print(count_paths(paths, 'you', 'out'))